# Solving ODEs Numerically Using an Explicit RK Scheme
We are going to learn how to solve and plot an ODE
$$y' = f(y)$$
in Julia.

The first thing we are going to do is learn why we can focus on 1st roder autonomous ODEs.

### Eliminating an explicit time variable t

If I have a non-autonomous ode 
$$z' = g(t,z)$$
then I can simply define 
$y = [t, z]$ and $f(y)=[1,g(y)]$
to convert to an autonous system with one more variable. 

### Eliminating Second Derivatives

If I have a second order ODE 
$$z'' = g(z,z')$$
in an $n$ dimensional vector variable $z$ then I can simply define 
$y = [z, z']$ and $f(y)=[y[n+1:2n],g(y)]$
to convert to a first order system with in vectors of length $2 n$. 

### Explicit Euler Scheme
The explicit Euler scheme just approximates the ODE 
$$y'=f(y)$$
over a step of length $h_n$ starting at $y_n$
with the slope (given by $f(y_n)$) 
at the start of the step.  This gives
$$
y_{n+1} = y_n + h_n f(y_n) 
$$
with $y_0$ defined by the initial condition. This is essentially
the *right hand rule* for approximating integrals from Calculus. The fanciest
integratiuon rule you learned in Calculus was probably *Simpson's Rule*

Euler's method is easy to implement.  Obvious questions are:
   - How big should $h_n$ be?
   - Is it worth doing something fancier?
 I guess we should implement Euler's method first.
 
 We have some questions to think about first:
  - How do we want to store the ys?
  - Do we want to explicitly store t?
  - Do we want to save the values of h?
  
 Answers: 
  - How do we want to store the ys?  2D Array size times MaxSteps
  - Do we want to explicitly store t?   Store t.
  - Do we want to save the values of h? Compute $h_n = t_{n+1}-t_{n}$

In [4]:
function EulerStep(f,y,h)
    k1=f(y)
    y + h*k1
end

EulerStep (generic function with 1 method)

### Explicit MidPoint Scheme
The explicit MidPoint scheme for the ODE 
$$y'=f(y)$$
over a step of length $h_n$ starting at $y_n$
first approximates the mid point $y(t_n + 0.5 h_n)$ using an explicit Euler Step
$$
y_{mid} = y_n + h_n 0.5 f(y_n)
$$
and then
approximates $y(t_n + h_n)$ using the slope at this approximate midpoint
$$
y_{n+1} = y_n + h_n f(y_{mid}). 
$$
This is an implementation of something like the *mid-point* rule in Calculus. 

In [5]:
function MidStep(f,y,h)
    k1 = f(y)
    z2 = y + 0.5*h*k1
    k2 = f(z2)
    y + h*k2
end

MidStep (generic function with 1 method)

### Explicit Trapezoid Scheme
The explicit Trapezoid scheme for the ODE 
$$y'=f(y)$$
over a step of length $h_n$ starting at $y_n$
approximates the end point $y(t_n + h_n)$ using an explicit Euler step
$$
y_{end} = y_n + h_n f(y_n)
$$
then
approximates $y(t_n + h_n)$ using the average of the slopes at the start and the slope at the end
$$
y_{n+1} = y_n + h_n (0.5 f(y_{n})+ 0.5 f(y_{end})) 
$$
This is an implementation of something like the *trapezoid* rule in Calculus. 

In [6]:
function TrapStep(f,y,h)
    k1 = f(y)
    z2 = y + 1.0*h*k1
    k2 = f(z2)
    y + h*(0.5*k1 + 0.5*k2)
end

TrapStep (generic function with 1 method)

### General Explicit RK scheme
A general RK scheme is defined by a strictly lower triangular matrix 
$A$ which specifies the sequential evaluations of the internal slopes
from the function $f$ and a vector $b$ which specifies how to 
average the slopes together to compute $y_{n+1}$. The size of the vector 
$b$ (or equivalently the square matrix $A$) is the number of internal stages and 
measures how complicated the scheme is.  

The standard 
terminology uses $z_i$ for the sequentially computed internal stages 
and $k_i = f(z_i)$ for the internal stage slopes through
$$
z_i = y_n + h_n \left( \sum_{j=1}^{s} a_{i,j} k_j \right)  
\quad
\mbox{with}
\quad
k_i = f(z_i)
$$
and the external step is computed from the stage slopes through
$$
y_{n+1} = y_n + h_n \sum_{j=1}^s b_j k_j
$$

In the real world solvers include two different $b$ vectors.  One is used to advance the scheme and the other is used to adjust the step size $h_n$ to control the error. 

We are going to write a solver for either the mid point or trapezoid rule and one of the rk schemes from the wiki page. 

Then we are going to write a general solver which performs a general RK step from a Butcher Tableau.  The question is how do we pass the coefficients in to the function.

As a structure. 